In [1]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
def load_from_gdrive(data_dir):
  # Initialize an empty list to store the DataFrames
  data_frames = []

  # Loop through each .csv file in the directory
  for dir in data_dir:
    for file_name in os.listdir(dir):
        if file_name.endswith('.csv'):
            file_path = os.path.join(dir, file_name)

            # Load the .csv file into a DataFrame
            df = pd.read_csv(file_path)

            # Remove the first column
            df = df.iloc[:, 4:]

            df  = df.div(360)

            df  = df.clip(upper =1)


            # Append the DataFrame to the list
            data_frames.append(df)

  return data_frames

In [4]:

# Define the directory path where the .csv files are located in Google Drive
data_dir = ['/content/drive/MyDrive/28 08 mod/Normal']

data_frames =  load_from_gdrive(data_dir)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd


# Determine the maximum length to establish the maximum timestep
max_length = max(len(df) for df in data_frames)

# Iterate over each data frame and apply truncation if necessary
timestep = 10  # Set timestep size
truncated_data_frames = []
truncated_data_frames = []
for df in data_frames:
    length = len(df)
    if length % timestep != 0:
        truncated_length = (length // timestep) * timestep  # Truncate to the nearest multiple of timestep
        truncated_df = df[:truncated_length]
        truncated_data_frames.append(truncated_df)
    else:
        truncated_data_frames.append(df)

train_data, test_data = train_test_split(truncated_data_frames, test_size=0.4)# 60 40

# Concatenate preprocessed data frames into a single tensor
train_sequences = [torch.Tensor(df.values) for df in train_data]
test_sequences = [torch.Tensor(df.values) for df in test_data]
# Function to make the buckets
def create_buckets(sequences, bucket_size):
    sequences.sort(key=len)
    buckets = []
    current_bucket = []
    max_length = 0
    for seq in sequences:
        if len(seq) > max_length + bucket_size:
            if current_bucket:
                buckets.append(current_bucket)
            current_bucket = []
            max_length = len(seq)
        current_bucket.append(seq)
    if current_bucket:
        buckets.append(current_bucket)
    return buckets

def collate_fn(batch):
    batch.sort(key=len, reverse=True)
    batch = pad_sequence(batch, batch_first=True)
    return batch


# Create buckets
bucket_size = 25  # Set bucket size (25)
train_buckets = create_buckets(train_sequences, bucket_size)
test_buckets = create_buckets(test_sequences, bucket_size)


batch_size = 1  # Only 1

# Create a DataLoader for each bucket
train_loaders = [DataLoader(bucket, batch_size=batch_size, shuffle=True, collate_fn=collate_fn) for bucket in train_buckets]
test_loaders = [DataLoader(bucket, batch_size=batch_size, shuffle=True, collate_fn=collate_fn) for bucket in test_buckets]

# Define the VAE LSTM network architecture
class VAE_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size,dropout_rate):
        super(VAE_LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.dropout_rate = dropout_rate

        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True,dropout = dropout_rate)
        self.fc_mu = nn.Linear(hidden_size, latent_size)
        self.fc_log_var = nn.Linear(hidden_size, latent_size)
        self.decoder = nn.LSTM(latent_size, hidden_size, batch_first=True,dropout = dropout_rate)
        self.fc = nn.Linear(hidden_size, input_size)

    def encode(self, x):
        _, (h_n, _) = self.encoder(x)
        h_n = h_n.squeeze()
        mu = self.fc_mu(h_n)
        log_var = self.fc_log_var(h_n)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        z = z.unsqueeze(0)
        h, _ = self.decoder(z)
        x_recon = self.fc(h.squeeze(0))
        return x_recon

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var


input_size = 59  # Number of features in a data frame
hidden_size = 256  #  hidden size
latent_size = 64  #  latent size

model = VAE_LSTM(input_size, hidden_size, latent_size,0.2)

# training loop
def train(model, loaders, optimizer, timestep, epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    reconstruction_loss = nn.MSELoss(reduction='sum')
    kl_divergence_loss = lambda mu, log_var: -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    for epoch in range(epochs):
        total_reconstruction_loss = 0
        total_kl_divergence_loss = 0
        total_loss = 0

        for loader in loaders:
            for batch in loader:
                x = batch.to(device)
                for i in range(0, len(x), timestep):
                    sub_sequence = x[:, i:i+timestep, :]
                    x_recon, mu, log_var = model(sub_sequence)

                    recon_loss = reconstruction_loss(x_recon, sub_sequence)
                    kl_loss = kl_divergence_loss(mu, log_var)
                    loss = recon_loss + kl_loss

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    total_reconstruction_loss += recon_loss.item()
                    total_kl_divergence_loss += kl_loss.item()
                    total_loss += loss.item()

        avg_reconstruction_loss = total_reconstruction_loss / sum(len(bucket) for bucket in train_buckets)
        avg_kl_divergence_loss = total_kl_divergence_loss / sum(len(bucket) for bucket in train_buckets)
        avg_loss = total_loss / sum(len(bucket) for bucket in train_buckets)

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Reconstruction Loss: {avg_reconstruction_loss:.4f}, "
              f"KL Divergence Loss: {avg_kl_divergence_loss:.4f}, "
              f"Total Loss: {avg_loss:.4f}")

# Train the model
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train(model, train_loaders, optimizer, timestep, epochs=100)

# Save the trained model
torch.save(model.state_dict(), 'trained_model.pt')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/100, Reconstruction Loss: 4.1486, KL Divergence Loss: 0.0559, Total Loss: 4.2045
Epoch 2/100, Reconstruction Loss: 3.6640, KL Divergence Loss: 0.0606, Total Loss: 3.7246
Epoch 3/100, Reconstruction Loss: 3.2720, KL Divergence Loss: 0.0629, Total Loss: 3.3349
Epoch 4/100, Reconstruction Loss: 2.8841, KL Divergence Loss: 0.0560, Total Loss: 2.9401
Epoch 5/100, Reconstruction Loss: 2.6155, KL Divergence Loss: 0.0569, Total Loss: 2.6723
Epoch 6/100, Reconstruction Loss: 2.3900, KL Divergence Loss: 0.0539, Total Loss: 2.4439
Epoch 7/100, Reconstruction Loss: 2.2901, KL Divergence Loss: 0.0497, Total Loss: 2.3398
Epoch 8/100, Reconstruction Loss: 2.0327, KL Divergence Loss: 0.0452, Total Loss: 2.0779
Epoch 9/100, Reconstruction Loss: 1.9360, KL Divergence Loss: 0.0331, Total Loss: 1.9692
Epoch 10/100, Reconstruction Loss: 1.7798, KL Divergence Loss: 0.0259, Total Loss: 1.8057
Epoch 11/100, Reconstruction Loss: 1.6807, KL Divergence Loss: 0.0229, Total Loss: 1.7035
Epoch 12/100, Recon

KeyboardInterrupt: ignored

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd





class VAE_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size,dropout_rate):
        super(VAE_LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.dropout_rate = dropout_rate

        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True,dropout = dropout_rate)
        self.fc_mu = nn.Linear(hidden_size, latent_size)
        self.fc_log_var = nn.Linear(hidden_size, latent_size)
        self.decoder = nn.LSTM(latent_size, hidden_size, batch_first=True,dropout = dropout_rate)
        self.fc = nn.Linear(hidden_size, input_size)

    def encode(self, x):
        _, (h_n, _) = self.encoder(x)
        h_n = h_n.squeeze()
        mu = self.fc_mu(h_n)
        log_var = self.fc_log_var(h_n)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        z = z.unsqueeze(0)
        h, _ = self.decoder(z)
        x_recon = self.fc(h.squeeze(0))
        return x_recon

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var


def train_with_early_stopping(model, train_loaders, val_loaders, optimizer, timestep, epochs, patience, beta):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_val_loss = float('inf')
    counter = 0

    reconstruction_loss = nn.MSELoss(reduction='sum')
    kl_divergence_loss = lambda mu, log_var: -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    for epoch in range(epochs):
        total_reconstruction_loss = 0
        total_kl_divergence_loss = 0
        total_loss = 0

        for loader in train_loaders:
            for batch in loader:
                x = batch.to(device)
                for i in range(0, len(x), timestep):
                    sub_sequence = x[:, i:i+timestep, :]
                    x_recon, mu, log_var = model(sub_sequence)

                    recon_loss = reconstruction_loss(x_recon, sub_sequence)
                    kl_loss = kl_divergence_loss(mu, log_var)
                    loss = recon_loss + beta * kl_loss

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    total_reconstruction_loss += recon_loss.item()
                    total_kl_divergence_loss += kl_loss.item()
                    total_loss += loss.item()

        avg_reconstruction_loss = total_reconstruction_loss / sum(len(loader) for loader in val_loaders)
        avg_kl_divergence_loss = total_kl_divergence_loss / sum(len(loader) for loader in val_loaders)
        avg_loss = total_loss / sum(len(loader) for loader in val_loaders)


        val_loss = evaluate(model, val_loaders, timestep, beta)

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Reconstruction Loss: {avg_reconstruction_loss:.4f}, "
              f"Train KL Divergence Loss: {avg_kl_divergence_loss:.4f}, "
              f"Train Total Loss: {avg_loss:.4f}, "
              f"Validation Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            #save_model(model, best_val_loss)
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping!")
                break

def evaluate(model, val_loaders, timestep, beta):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    total_reconstruction_loss = 0
    total_kl_divergence_loss = 0
    total_loss = 0

    reconstruction_loss = nn.MSELoss(reduction='sum')
    kl_divergence_loss = lambda mu, log_var: -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    with torch.no_grad():
        for loader in val_loaders:
            for batch in loader:
                x = batch.to(device)
                for i in range(0, len(x), timestep):
                    sub_sequence = x[:, i:i+timestep, :]
                    x_recon, mu, log_var = model(sub_sequence)

                    recon_loss = reconstruction_loss(x_recon, sub_sequence)
                    kl_loss = kl_divergence_loss(mu, log_var)
                    loss = recon_loss + beta * kl_loss

                    total_reconstruction_loss += recon_loss.item()
                    total_kl_divergence_loss += kl_loss.item()
                    total_loss += loss.item()

    avg_reconstruction_loss = total_reconstruction_loss / sum(len(loader) for loader in val_loaders)
    avg_kl_divergence_loss = total_kl_divergence_loss / sum(len(loader) for loader in val_loaders)
    avg_loss = total_loss / sum(len(loader) for loader in val_loaders)


    return avg_loss


def train_with_early_stopping(model, train_loaders, val_loaders, optimizer, timestep, epochs, patience, beta):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_val_loss = float('inf')
    counter = 0

    reconstruction_loss = nn.MSELoss(reduction='sum')
    kl_divergence_loss = lambda mu, log_var: -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    for epoch in range(epochs):
        total_reconstruction_loss = 0
        total_kl_divergence_loss = 0
        total_loss = 0

        for loader in train_loaders:
            for batch in loader:
                x = batch.to(device)
                for i in range(0, len(x), timestep):
                    sub_sequence = x[:, i:i+timestep, :]
                    x_recon, mu, log_var = model(sub_sequence)

                    recon_loss = reconstruction_loss(x_recon, sub_sequence)
                    kl_loss = kl_divergence_loss(mu, log_var)
                    loss = recon_loss + beta * kl_loss

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    total_reconstruction_loss += recon_loss.item()
                    total_kl_divergence_loss += kl_loss.item()
                    total_loss += loss.item()

        avg_reconstruction_loss = total_reconstruction_loss / sum(len(bucket) for bucket in train_buckets)
        avg_kl_divergence_loss = total_kl_divergence_loss / sum(len(bucket) for bucket in train_buckets)
        avg_loss = total_loss / sum(len(bucket) for bucket in train_buckets)

        val_loss = evaluate(model, val_loaders, timestep, beta)

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Reconstruction Loss: {avg_reconstruction_loss:.4f}, "
              f"Train KL Divergence Loss: {avg_kl_divergence_loss:.4f}, "
              f"Train Total Loss: {avg_loss:.4f}, "
              f"Validation Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            #save_model(model, best_val_loss)
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping!")
                break

def evaluate(model, val_loaders, timestep, beta):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    total_reconstruction_loss = 0
    total_kl_divergence_loss = 0
    total_loss = 0

    reconstruction_loss = nn.MSELoss(reduction='sum')
    kl_divergence_loss = lambda mu, log_var: -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    with torch.no_grad():
        for loader in val_loaders:
            for batch in loader:
                x = batch.to(device)
                for i in range(0, len(x), timestep):
                    sub_sequence = x[:, i:i+timestep, :]
                    x_recon, mu, log_var = model(sub_sequence)

                    recon_loss = reconstruction_loss(x_recon, sub_sequence)
                    kl_loss = kl_divergence_loss(mu, log_var)
                    loss = recon_loss + beta * kl_loss

                    total_reconstruction_loss += recon_loss.item()
                    total_kl_divergence_loss += kl_loss.item()
                    total_loss += loss.item()

    avg_reconstruction_loss = total_reconstruction_loss / sum(len(bucket) for bucket in val_buckets)
    avg_kl_divergence_loss = total_kl_divergence_loss / sum(len(bucket) for bucket in val_buckets)
    avg_loss = total_loss / sum(len(bucket) for bucket in val_buckets)

    return avg_loss

def save_model(model, val_loss,learning_rate,drop,hidden_size,latent_size):
    #drive.mount('/content/drive')
    model_name = f"{val_loss:.4f}_hidden{hidden_size}_latent{latent_size}_timestep{timestep}_learning_rate{learning_rate}_dropout{drop}.pt"
    model_path = os.path.join("/content/drive/MyDrive/VAE_LSTM10", model_name)
    torch.save(model.state_dict(), model_path)


In [6]:
def grid_search(train_loader, val_loader, hidden_sizes, latent_sizes, learning_rates, timestep, epochs, patience,dropout):
    for hidden_size in hidden_sizes:
        for latent_size in latent_sizes:
            for learning_rate in learning_rates:
              for drop in dropout:

                  print(f'Current params: hidden_size={hidden_size}, latent_size={latent_size}, learning_rate={learning_rate}, drop = {drop}')
                  model = VAE_LSTM(input_size, hidden_size, latent_size,dropout_rate = drop)
                  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                  train_with_early_stopping(model, train_loader, val_loader, optimizer, timestep, epochs, patience,beta = 1)
                  val_loss = evaluate(model, val_loader, timestep,beta = 1)
                  save_model(model, val_loss, learning_rate,drop,hidden_size,latent_size)


In [7]:

# Determine the maximum length
max_length = max(len(df) for df in data_frames)

# Iterate over each data frame and apply truncation if necessary
timestep = 10  # Set timestep size
truncated_data_frames = []
for df in data_frames:
    length = len(df)
    if length % timestep != 0:
        truncated_length = (length // timestep) * timestep  # Truncate to the nearest multiple of timestep
        truncated_df = df[:truncated_length]
        truncated_data_frames.append(truncated_df)
    else:
        truncated_data_frames.append(df)

# Concatenate preprocessed data frames into a single tensor
sequences = [torch.Tensor(df.values) for df in truncated_data_frames]

# Function to make the buckets
def create_buckets(sequences, bucket_size):
    sequences.sort(key=len)
    buckets = []
    current_bucket = []
    max_length = 0
    for seq in sequences:
        if len(seq) > max_length + bucket_size:
            if current_bucket:
                buckets.append(current_bucket)
            current_bucket = []
            max_length = len(seq)
        current_bucket.append(seq)
    if current_bucket:
        buckets.append(current_bucket)
    return buckets

def collate_fn(batch):
    batch.sort(key=len, reverse=True)
    batch = pad_sequence(batch, batch_first=True)
    return batch


# Create buckets
bucket_size = 25  # Set bucket size (25)
buckets = create_buckets(sequences, bucket_size)

from sklearn.model_selection import train_test_split

# Split the buckets into training and validation sets
train_buckets, val_buckets = train_test_split(buckets, test_size=0.4, random_state=42)
val_buckets, test_buckets = train_test_split(val_buckets, test_size=0.5, random_state=42)



batch_size = 1  # Only 1

# Create a DataLoader for each bucket
train_loaders = [DataLoader(bucket, batch_size=batch_size, shuffle=True, collate_fn=collate_fn) for bucket in train_buckets]
val_loaders = [DataLoader(bucket, batch_size=batch_size, shuffle=False, collate_fn=collate_fn) for bucket in val_buckets]

hidden_sizes = [128,256, 512]
latent_sizes = [128,256,512]
learning_rates = [0.00001, 0.0001]
dropout = [0,0.2,0.4]
epochs = 100
patience = 20
input_size=59

grid_search(train_loaders,val_loaders, hidden_sizes, latent_sizes, learning_rates,timestep, epochs, patience,dropout)

Current params: hidden_size=128, latent_size=128, learning_rate=1e-05, drop = 0


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 10, 59])) that is different to the input size (torch.Size([59])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, Train Reconstruction Loss: 7.5383, Train KL Divergence Loss: 0.3185, Train Total Loss: 7.8568, Validation Loss: 7.4266
Epoch 2/100, Train Reconstruction Loss: 7.3733, Train KL Divergence Loss: 0.3050, Train Total Loss: 7.6782, Validation Loss: 7.4881
Epoch 3/100, Train Reconstruction Loss: 7.3456, Train KL Divergence Loss: 0.2931, Train Total Loss: 7.6388, Validation Loss: 7.5841
Epoch 4/100, Train Reconstruction Loss: 7.5613, Train KL Divergence Loss: 0.2824, Train Total Loss: 7.8437, Validation Loss: 7.7745
Epoch 5/100, Train Reconstruction Loss: 7.2727, Train KL Divergence Loss: 0.2728, Train Total Loss: 7.5455, Validation Loss: 7.4910
Epoch 6/100, Train Reconstruction Loss: 7.2649, Train KL Divergence Loss: 0.2638, Train Total Loss: 7.5288, Validation Loss: 7.3347
Epoch 7/100, Train Reconstruction Loss: 7.1345, Train KL Divergence Loss: 0.2552, Train Total Loss: 7.3896, Validation Loss: 7.6009
Epoch 8/100, Train Reconstruction Loss: 7.3656, Train KL Divergence Loss: 0.

KeyboardInterrupt: ignored